<a href="https://colab.research.google.com/github/NandiniLReddy/Teaching_AgenticDesign101/blob/main/PromptChaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Chaining implementation

## background info

implementing prompt chaining requires direct to seqential function calls within the scripts; so some specialized frameworks like control, flow, state and component integration is requried:

so some frameworks like langchain, langgraph, crew ai, and google ADK provides such environment:

## langchain
so langchain and lang-graph are suitable choices as their core apis are explicitly designed for composing chains and graph operations.
## langgraph
langchain provides foundational abstractions for linear sequences,
while langgraph extends these capabilities to support stateful and cyclical computations which are necessary for implementing more sophisticated agentic behaviours

### this is the implementation of 2 step prompt chaining that functions as a data processing pipeline

## initial stage  --  parse unstrcutured text and extract specific information
## subsequent stage - receives this structured output and transforms into a structured data format

## start

In [1]:
pip install langchain langchain-community langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
#load llm
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [10]:
#prompt-1: Extract info
prompt_extract = ChatPromptTemplate.from_template("Trandform the following specifications from the following text: \n\n{text_input}")

In [11]:
#prompt-2: Transform to JSON
prompt_transform = ChatPromptTemplate.from_template("Transform the following text into a JSON object with 'cpu', 'memory', and 'storage' as keys: \n\n{specifications}")

In [12]:
#building chain using LCEL(Langchain expression language)
extraction_chain = prompt_extract | llm | StrOutputParser()
#the StrOutputParser() converts the LLM's message output into a simple string.

In [14]:
full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)
#the full chain passes the output of extraction chain into the 'specifications'

In [15]:
#let's run
input_text = "The new laptop model features a 3.5 Ghz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

In [16]:
final_result = full_chain.invoke({"text_input": input_text})

In [17]:
print(final_result)

{
    "cpu": "3.5 Ghz octa-core",
    "memory": "16GB",
    "storage": "1TB NVMe SSD"
}
